In [1]:
import sys
!pip install awswrangler -q

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import awswrangler as wr

In [4]:

# test if you can read from a private bucket
df = wr.s3.read_csv('s3://techcatalyst-raw/stocks/GOOG.csv')
df.head()

,Date,Open,High,Low,Close,Volume
0,1/2/2025 16:00:00,191.49,193.20,188.71,190.63,17545162
1,1/3/2025 16:00:00,192.73,194.50,191.35,193.13,12874957
2,1/6/2025 16:00:00,195.15,199.56,195.06,197.96,19483323
3,1/7/2025 16:00:00,198.27,202.14,195.94,196.71,16966760
4,1/8/2025 16:00:00,193.95,197.64,193.75,195.39,14335341


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    140 non-null    object 
 1   Open    140 non-null    float64
 2   High    140 non-null    float64
 3   Low     140 non-null    float64
 4   Close   140 non-null    float64
 5   Volume  140 non-null    int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 6.7+ KB


In [6]:
databases = wr.catalog.databases()
print(databases)

               Database       Description
0             aamnah_db                  
1           aamnah_taxi                  
2             alexia_db                  
3           alexia_logs                  
4           alexia_song                  
5           alexia_taxi                  
6      awswrangler_test                  
7                ben_db                  
8              ben_song                  
9              ben_taxi                  
10           blake_taxi                  
11          blake_wr_db                  
12    camrenn.rivera_db                  
13  camrenn.rivera_taxi                  
14              db_name                  
15              default  default database
16              emma_db                  
17            emma_taxi                  
18           fabiola_db                  
19         fabiola_taxi                  
20           jaden_taxi                  
21        jadenastle_db                  
22           melissa_db           

In [7]:
name = 'aamnah'
database_name = f"{name}_db"
wr.catalog.create_database(database_name)

databases = wr.catalog.databases()
print(databases)

AlreadyExists: Database aamnah_db already exists and <exist_ok> is set to False.

In [8]:
wr.catalog.tables(database=database_name)

,Database,Table,Description,TableType,Columns,Partitions
0,aamnah_db,aamnah_stock,,EXTERNAL_TABLE,"date, open, high, low, close, volume",


In [9]:
wr.s3.to_parquet(
    df=df, # the DataFrame you just created 
    path=f"s3://techcatalyst-raw/{name}/", # write to the techcatalyst-raw bucket under your folder name (or it would create a new folder if it does not exist)
    dataset=True, 
    database='aamnah_db', # the name of the database you just created in AWS Glue 
    table='aamnah_stock', # pick a table name for example YOURNAME_STOCK
    mode='overwrite'
    )

{'paths': ['s3://techcatalyst-raw/aamnah/43f83cf675004457ae43fd5a8f9fdfeb.snappy.parquet'],
 'partitions_values': {}}

In [10]:
wr.catalog.tables(database = database_name)

,Database,Table,Description,TableType,Columns,Partitions
0,aamnah_db,aamnah_stock,,EXTERNAL_TABLE,"date, open, high, low, close, volume",


In [11]:
wr.catalog.tables(name_contains="stock")

,Database,Table,Description,TableType,Columns,Partitions
0,aamnah_db,aamnah_stock,,EXTERNAL_TABLE,"date, open, high, low, close, volume",
1,alexia_db,alexia_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
2,ben_db,ben_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
3,blake_wr_db,blake_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
4,camrenn.rivera_db,camrenn_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
5,emma_db,emma_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
6,fabiola_db,fabiola_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
7,jadenastle_db,jaden_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
8,melissa_db,melissa_stocks,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
9,michael_db,michael_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",


In [12]:
df = wr.s3.read_parquet_table(database='aamnah_db',table='aamnah_stock')
# Display the DataFrame's first few rows
df.head()

,date,open,high,low,close,volume
0,1/2/2025 16:00:00,191.49,193.20,188.71,190.63,17545162
1,1/3/2025 16:00:00,192.73,194.50,191.35,193.13,12874957
2,1/6/2025 16:00:00,195.15,199.56,195.06,197.96,19483323
3,1/7/2025 16:00:00,198.27,202.14,195.94,196.71,16966760
4,1/8/2025 16:00:00,193.95,197.64,193.75,195.39,14335341


In [13]:
table_details = wr.catalog.get_tables(database=database_name)
next(table_details)

{'Name': 'aamnah_stock',
 'DatabaseName': 'aamnah_db',
 'CreateTime': datetime.datetime(2025, 8, 4, 20, 18, 25, tzinfo=tzlocal()),
 'UpdateTime': datetime.datetime(2025, 8, 6, 13, 32, 59, tzinfo=tzlocal()),
 'Retention': 0,
 'StorageDescriptor': {'Columns': [{'Name': 'date', 'Type': 'string'},
   {'Name': 'open', 'Type': 'double'},
   {'Name': 'high', 'Type': 'double'},
   {'Name': 'low', 'Type': 'double'},
   {'Name': 'close', 'Type': 'double'},
   {'Name': 'volume', 'Type': 'bigint'}],
  'Location': 's3://techcatalyst-raw/aamnah/',
  'InputFormat': 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat',
  'OutputFormat': 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat',
  'Compressed': True,
  'NumberOfBuckets': -1,
  'SerdeInfo': {'SerializationLibrary': 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe',
   'Parameters': {'serialization.format': '1'}},
  'BucketColumns': [],
  'SortColumns': [],
  'Parameters': {'CrawlerSchemaDeserializerVersion'

In [14]:
# Example adding additional metadata information 

desc = "This is my stock table."
param = {"source": "Google", "class": "stock"}
comments = {
    "Date": "Trading Date",
    "Open": "Opening Price",
    "Close": "Closing Price"
}

wr.s3.to_parquet(
    df=df,
    path='s3://techcatalyst-raw/aamnah', # CHANGE THIS TO USE YOUR NAME for example s3://TECHCATALYST/TATWAN
    dataset=True,
    database=database_name,
    table='aamnah_stock',
    mode='overwrite',
    glue_table_settings=wr.typing.GlueTableSettings(description=desc,  # here we are passing some metadata
                                                    parameters=param, 
                                                    columns_comments=comments),
    )

{'paths': ['s3://techcatalyst-raw/aamnah/1b7ae79248e7430692eff96187474936.snappy.parquet'],
 'partitions_values': {}}

In [15]:
wr.catalog.table(database=database_name, table='aamnah_stock')

,Column Name,Type,Partition,Comment
0,date,string,False,Trading Date
1,open,double,False,Opening Price
2,high,double,False,
3,low,double,False,
4,close,double,False,Closing Price
5,volume,bigint,False,


In [16]:
wr.s3.list_objects('s3://techcatalyst-raw/stage/')

['s3://techcatalyst-raw/stage/yellow_tripdata.csv',
 's3://techcatalyst-raw/stage/yellow_tripdata.json',
 's3://techcatalyst-raw/stage/yellow_tripdata.parquet']

In [17]:
wr.s3.download(path='s3://techcatalyst-raw/stocks/GOOG.csv', 
               local_file='./new_file.csv')

In [18]:
your_name = 'aamnah'
file_name = 'GOOG_NEW.csv'
wr.s3.upload(local_file='new_file.csv',path= f's3://techcatalyst-raw/{your_name}/uploads/{file_name}')

In [19]:
wr.s3.list_objects(f's3://techcatalyst-raw/{your_name}/uploads/')

['s3://techcatalyst-raw/aamnah/uploads/GOOG_NEW.csv']

In [20]:
# A name for the new Glue Database it should be YOURNAME_TAXI
db_name =  'AAMNAH_TAXI'

# A name for the table it should be YOURNAME_TRIPDATA
table_name =  'AAMNAH_TRIPDATA'

# the path_direcoty to should point to the bucket (main directory)
s3_path_directory = 's3://techcatalyst-raw/'

# The path_file should be the full path to the actual file
s3_path_file = 's3://techcatalyst-raw/taxi_data'


In [21]:
# uncomment below if you ran into issues to clean things up and rerun the cell
wr.catalog.delete_table_if_exists(database=db_name, table=table_name) 

# Create the new Glue database first based on the db_name you created
# wr.catalog.create_database(db_name)
wr.catalog.create_database(name='AAMNAH_TAXI', exist_ok=True)
                           
    
# This function can extract the schema from our file and returns a tuple: (schema, partitions). We only need the schema. 
columns_types, partitions_types = wr.s3.read_parquet_metadata(path=s3_path_file)
print("Successfully read schema from Parquet file.")

Successfully read schema from Parquet file.


In [22]:

wr.catalog.create_parquet_table(
    database=db_name, # pass the database name
    table=table_name, # pass the table name
    path=s3_path_directory, # use the directoy here 
    columns_types=columns_types,  # Pass the schema here
    partitions_types=partitions_types
)
print(f"Table '{table_name}' created successfully in database '{db_name}'.")

Table 'AAMNAH_TRIPDATA' created successfully in database 'AAMNAH_TAXI'.


In [24]:
your_name = 'aamnah'
file_name = 'test.csv'
wr.s3.upload(local_file='test.csv',path= f's3://techcatalyst-raw/{your_name}/{file_name}')